In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import random
np.random.seed(1991)
from multiprocessing import Pool
seed_list=list(range(1,1000,100))
import numba
import glob

In [2]:
@numba.jit(nopython=True)
def year_binner(year,val=10):
    return year - year%val


@numba.jit(nopython=True)
def decremover(dc):

    new_constituents=[]
    for constituent,decades in dc.items():
        if len(decades)==1:
            if 2000 in decades:
                new_constituents.append(constituent)
    return(new_constituents)

In [3]:
modifier_list=[]
with open('/data/dharp/compounds/datasets/modifier_list.txt','r') as f:
        modifier_list=f.read().split("\n")
modifier_list=modifier_list[:-1]

In [4]:
head_list=[]
with open('/data/dharp/compounds/datasets/head_list.txt','r') as f:
        head_list=f.read().split("\n")
head_list=head_list[:-1]
head_list[-10:]

['rostow',
 'schroeder',
 'taintor',
 'taussig',
 'wartofsky',
 'winnicott',
 '6ha',
 'gasset',
 'jst',
 'friedenberg']

In [5]:
    comp_list = [pd.read_pickle(filename) for filename in glob.glob('/data/dharp/compounds/datasets/compounds/*.pkl')]        

    compounds=pd.concat(comp_list)

In [6]:
compounds.year=compounds.year.astype("int32")
compounds['time']=year_binner(compounds['year'].values,10)
compounds=compounds.groupby(['modifier','head','time'])['count'].sum().to_frame()
compounds.reset_index(inplace=True)
compounds.modifier=compounds.modifier.replace(r'_(NOUN|PROPN)$', r'', regex=True)
compounds['head']=compounds['head'].replace(r'_(NOUN|PROPN)$', r'', regex=True)
compounds=compounds.loc[compounds.modifier.isin(modifier_list)&compounds['head'].isin(head_list)]
compounds

,modifier,head,time,count
0,#,#,1810,1
1,#,#,1850,1
2,#,#,1880,1
3,#,#,1910,1
4,#,#,1950,1
...,...,...,...,...
13088292,♦,research,1980,32
13088293,♦,research,1990,8
13088294,♦,research,2000,1
13088295,♦,tip,1990,23


In [7]:
novel_compounds=compounds.loc[compounds.groupby(['modifier','head'])['time'].transform('count').eq(1)]
novel_compounds_2000=novel_compounds.loc[novel_compounds.time==2000]
novel_compounds_2000_list=list(zip(novel_compounds_2000['modifier'].tolist(),novel_compounds_2000['head'].tolist()))
display(len(novel_compounds_2000_list))

12572

In [8]:
novel_compounds_2000['count'].describe()

count    12572.000
mean       201.720
std       2145.420
min         40.000
25%         63.000
50%        105.000
75%        189.000
max     235711.000
Name: count, dtype: float64

heads=pd.read_csv("/data/dharp/compounds/datasets/v2_aware/heads_reduced.csv",sep="\t",header=None)
heads

modifiers=pd.read_csv("/data/dharp/compounds/datasets/v2_aware/modifiers_reduced.csv",sep="\t",header=None)
modifiers

modifiers[1].value_counts()

novel_compounds_2000.drop(['time','count'],axis=1).merge(modifiers.loc[modifiers[1]==1970],left_on=['modifier'],right_on=0,how='left')

In [9]:
maske_list=novel_compounds.loc[novel_compounds.time==1990]
maske_list=list(zip(maske_list['modifier'].tolist(),maske_list['head'].tolist()))
display(len(maske_list))
maske_list[110:130]

4685

[('aids', 'alert'),
 ('aids', 'american'),
 ('aids', 'bibliography'),
 ('aids', 'blood'),
 ('aids', 'sample'),
 ('air', 'breakdown'),
 ('air', 'bump'),
 ('air', 'dispersion'),
 ('air', 'shipment'),
 ('air', 'emissions'),
 ('air', 'tables'),
 ('aircraft', 'attitude'),
 ('aircraft', 'axis'),
 ('aircraft', 'body'),
 ('aircraft', 'campaign'),
 ('aircraft', 'correlation'),
 ('aircraft', 'dc'),
 ('aircraft', 'environment'),
 ('aircraft', 'mission'),
 ('aircraft', 'motion')]

In [10]:
novel_compounds_new=compounds.loc[compounds.groupby(['modifier','head'])['time'].transform('count').eq(2)]
novel_compounds_1990=novel_compounds_new.loc[novel_compounds_new.time.isin([1990,2000])]
novel_compounds_1990_list=list(zip(novel_compounds_1990['modifier'].tolist(),novel_compounds_1990['head'].tolist()))
novel_compounds_1990_list=list(set(novel_compounds_1990_list))
display(len(novel_compounds_1990_list))
novel_compounds_1990_list[:10]

61474

[('presbyterian', 'woman'),
 ('learner', 'discourse'),
 ('man', 'relations'),
 ('public', 'competition'),
 ('iff', 'k'),
 ('american', 'flea'),
 ('columbia', 'first'),
 ('terry', 'college'),
 ('tour', 'lady'),
 ('rama', 'stories')]

In [11]:
total_compounds=compounds.query('1950 <= time <= 1989').copy()
total_compounds_list=list(zip(total_compounds['modifier'].tolist(),total_compounds['head'].tolist()))
total_compounds_list=list(set(total_compounds_list))
display(len(total_compounds_list))
total_compounds_list[:10]

883875

[('m.', 'tests'),
 ('party', 'desire'),
 ('site', 'hotel'),
 ('mr.', 'luce'),
 ('national', 'free'),
 ('slit', 'a'),
 ('fact', 'rome'),
 ('value', 'b.'),
 ('household', 'audio'),
 ('adult', 'tick')]

In [12]:
total_compounds_list=list(set(total_compounds_list).difference(novel_compounds_1990_list))
display(len(total_compounds_list))
total_compounds_list[:10]

878149

[('m.', 'tests'),
 ('party', 'desire'),
 ('site', 'hotel'),
 ('mr.', 'luce'),
 ('national', 'free'),
 ('slit', 'a'),
 ('fact', 'rome'),
 ('value', 'b.'),
 ('household', 'audio'),
 ('fertilizer', 'relative')]

In [13]:
compound_list=novel_compounds_1990_list+novel_compounds_2000_list+total_compounds_list

In [14]:
len(set(total_compounds_list).intersection(novel_compounds_2000_list))

0

In [15]:
len(set(novel_compounds_2000_list).intersection(novel_compounds_1990_list))

0

In [16]:
len(set(total_compounds_list).intersection(novel_compounds_1990_list))

0

In [17]:
combined_list=[x+' '+y for x,y in novel_compounds_2000_list]

In [18]:
with open('/data/dharp/compounds/datasets/compound_trainset.txt','w') as f:
    for head,mod in total_compounds_list:
        f.write(f'{head} {mod}\n')

#pkl.dump(total_compounds_list,open('compound_trainset.pkl','wb'))

In [19]:
with open('/data/dharp/compounds/datasets/compound_validset.txt','w') as f:
    for head,mod in novel_compounds_1990_list:
        f.write(f'{head} {mod}\n')


#pkl.dump(novel_compounds_1990_list,open('compound_validset.pkl','wb'))

In [20]:
with open('/data/dharp/compounds/datasets/compound_testset.txt','w') as f:
    for head,mod in novel_compounds_2000_list:
        f.write(f'{head} {mod}\n')


#pkl.dump(novel_compounds_2000_list,open('compound_testset.pkl','wb'))

In [21]:
combined_list[-10:]

['• reliability',
 '• safety',
 '• shadow',
 '• shape',
 '• shop',
 '• switzerland',
 '• transportation',
 '• type',
 '• water',
 '™ grade']

In [22]:
returned_list=[x.split(' ') for x in combined_list]
returned_list[:10]

[['#', 'husky'],
 ['#', 'nat'],
 ['#', 'new'],
 ['#', 'region'],
 ['#', 'text'],
 ['%', 'advance'],
 ['%', 'cluster'],
 ['%', 'household'],
 ['%', 'lifetime'],
 ['%', 'purchase']]

## Creation of Negative Classes - CorruptHead and CorruptModifier


In [25]:
def corrupt_dataset_creator(choice):
    random.seed(choice)
    corrupt_modifier_list=[]
    corrupt_head_list=[]
    for cur_num,(modifier,head) in enumerate(returned_list):
    # CorruptHead , Modifier is same
        #print(cur_num)

        while True:
            temp_corrupt_head=random.choice(head_list)
            if (modifier,temp_corrupt_head) not in compound_list:
                corrupt_head_list.append((modifier,temp_corrupt_head))
                break
    # CorruptModifier , Head is same
        while True:
            temp_corrupt_modifier=random.choice(modifier_list)
            if (temp_corrupt_modifier,head) not in compound_list:
                corrupt_modifier_list.append((temp_corrupt_modifier,head))
                break
    return corrupt_head_list,corrupt_modifier_list

In [26]:
pool = Pool(10)
test_results=pool.map(corrupt_dataset_creator,seed_list)

In [37]:
for num,(corrupt_heads,corrupt_modifiers) in enumerate(test_results):
    print(num+1)
    pkl.dump( corrupt_heads, open( '/data/dharp/compounds/datasets/corrupt_heads_'+str(num+1)+'.pkl', "wb" ) )
    
    pkl.dump( corrupt_modifiers, open( '/data/dharp/compounds/datasets/corrupt_modifiers_'+str(num+1)+'.pkl', "wb" ) )

1
2
3
4
5
6
7
8
9
10


In [36]:
returned_list[300:320]

[['affective', 'functions'],
 ['affective', 'intelligence'],
 ['afferent', 'target'],
 ['afghan', 'jihad'],
 ['africa', 'gain'],
 ['africa', 'war'],
 ['african', 'arena'],
 ['african', 'cinema'],
 ['african', 'exports'],
 ['african', 'farmers'],
 ['african', 'farms'],
 ['african', 'film'],
 ['african', 'imagination'],
 ['african', 'intelligentsia'],
 ['african', 'livestock'],
 ['african', 'logic'],
 ['african', 'medicines'],
 ['african', 'poverty'],
 ['african', 'self'],
 ['african', 'studio']]

In [35]:
test_results[0][0][300:320]

[('affective', 'forwarding'),
 ('affective', 'phc'),
 ('afferent', 'imperfection'),
 ('afghan', 'diversion'),
 ('africa', 'darkness'),
 ('africa', 'gram'),
 ('african', 'volcanism'),
 ('african', 'man'),
 ('african', 'kibbutz'),
 ('african', 'mould'),
 ('african', 'steroid'),
 ('african', 'tensor'),
 ('african', 'henderson'),
 ('african', 'snap'),
 ('african', 'rus'),
 ('african', 'toxicity'),
 ('african', 'bedivere'),
 ('african', 'rappaccini'),
 ('african', 'refuge'),
 ('african', 'alstyne')]

In [34]:
test_results[0][1][300:320]

[('eucharistic', 'functions'),
 ('rot', 'intelligence'),
 ('patrick', 'target'),
 ('polylinker', 'jihad'),
 ('leaf', 'gain'),
 ('phil', 'war'),
 ('arcuate', 'arena'),
 ('anymouse', 'cinema'),
 ('ovarian', 'exports'),
 ('marmota', 'farmers'),
 ('rectus', 'farms'),
 ('pink', 'film'),
 ('proportionality', 'imagination'),
 ('cusip', 'intelligentsia'),
 ('riemann', 'livestock'),
 ('stigma', 'logic'),
 ('pinocytosis', 'medicines'),
 ('f8', 'poverty'),
 ('ecologic', 'self'),
 ('manor', 'studio')]

In [29]:
test_results[1][0][:20]

[('#', 'rs'),
 ('#', 'jsm'),
 ('#', 'wastage'),
 ('#', 'melatonin'),
 ('#', 'vargas'),
 ('%', 'timber'),
 ('%', 'santos'),
 ('%', 'garfield'),
 ('%', 'therapist'),
 ('%', 'ogg'),
 ('*', 'daishi'),
 ('+', 'charities'),
 ('+', 'barons'),
 ('-', 'harvester'),
 ('-', 'mammogram'),
 ('-', 'procession'),
 ('-', 'horsemen'),
 ('-', 'somatosensory'),
 ('-', 'real'),
 ('-', 'hemp')]